In [3]:
#Left Joins nonstandard
import pandas as pd
import os
import configparser
import sys
sys.path.append('/project/Development')
from helpers import general_helpers
import configparser
config = configparser.ConfigParser()
config.read('/project/Development/config.ini')
host = config['DATABASE']['HOST']
username = config['DATABASE']['USERNAME']
password = config['DATABASE']['PASSWORD']
new_database = config['DATABASE']['NEW_DB']
old_database = config['DATABASE']['OLD_DB']
temporary_upload = config['DATABASE']['TEMP_UPLOAD_DB']
previous_qa_loc = config['FOLDERS']['OLD_QA_LOC']
new_qa_loc = config['FOLDERS']['NEW_QA_LOC']
latest_expected_date = config['CONSTANTS']['LATEST_DATE']

engine = general_helpers.connect_to_db(host, username, password, new_database)

In [5]:
query_list=["SELECT count(distinct patent_id) FROM rawassignee LEFT JOIN patent on patent.id = patent_id where patent.id is null;",
            "SELECT count(distinct patent_id) FROM rawassignee LEFT JOIN patent on patent.id = patent_id where patent.id is null;",
            "SELECT count(pct_data.patent_id) FROM pct_data LEFT JOIN patent on patent.id = pct_data.patent_id where patent.id is null;",
            "SELECT count(distinct rawlocation.location_id) FROM location_assignee LEFT JOIN rawlocation on rawlocation.location_id_transformed = location_assignee.location_id LEFT JOIN location on location.id = rawlocation.location_id where assignee_id = '';",
            "SELECT count(*) FROM cpc_current LEFT JOIN cpc_subsection on cpc_subsection.id = cpc_current.subsection_id where id is null limit 100;",
            "SELECT count(distinct location_assignee.location_id) FROM temp_patent_list LEFT JOIN patent_assignee on patent_assignee.patent_id = temp_patent_list.id LEFT JOIN location_assignee on location_assignee.assignee_id = patent_assignee.assignee_id LEFT JOIN rawlocation on rawlocation.location_id_transformed = location_assignee.location_id;","SELECT count(distinct location_inventor.location_id) FROM temp_patent_list LEFT JOIN patent_inventor on patent_inventor.patent_id = temp_patent_list.id LEFT JOIN location_inventor on location_inventor.inventor_id = patent_inventor.inventor_id LEFT JOIN location on location.id = location_inventor.location_id where location.id is null;","SELECT count(distinct patent_id) FROM rawinventor LEFT JOIN rawlocation on rawlocation.id = rawinventor.rawlocation_id where rawlocation.id is null;",
            "SELECT count(distinct patent_id) FROM cpc_current LEFT JOIN cpc_subgroup on cpc_subgroup.id = cpc_current.subgroup_id where id is null;",
            "SELECT count(distinct mainclass_id) FROM uspc_current LEFT JOIN mainclass_current on mainclass_current.id = uspc_current.mainclass_id where title is null AND mainclass_id != 'No longer published';",
            "SELECT count(distinct id) FROM location_inventor LEFT JOIN location on location.id = location_inventor.location_id where location_inventor.location_id is null;","SELECT count(distinct patent_id) FROM nber LEFT JOIN nber_category on nber_category.id = nber.category_id where title is null;","SELECT count(distinct category_id) FROM nber LEFT JOIN nber_category on nber_category.id = nber.category_id where title is null;",
"SELECT count(distinct patent_id) FROM nber LEFT JOIN nber_subcategory on nber_subcategory.id = nber.subcategory_id where title is null;","SELECT count(distinct subcategory_id) FROM nber LEFT JOIN nber_subcategory on nber_subcategory.id = nber.subcategory_id where title is null;","SELECT count(distinct patent_id) FROM uspc LEFT JOIN mainclass on mainclass.id = uspc.mainclass_id where mainclass.id is null;",
            "SELECT count(distinct mainclass_id) FROM uspc LEFT JOIN mainclass on mainclass.id = uspc.mainclass_id where patent_id is null;","SELECT count(distinct patent_id) FROM uspc LEFT JOIN subclass on subclass.id = uspc.mainclass_id where subclass.id is null;",
            "SELECT count(distinct subclass_id) FROM uspc LEFT JOIN subclass on subclass.id = uspc.mainclass_id where patent_id is null;","SELECT count(distinct wipo.patent_id) FROM wipo LEFT JOIN wipo_field on wipo_field.id = wipo.field_id where wipo_field.id is null;",
            "SELECT count(distinct patent_id) FROM cpc_current LEFT JOIN cpc_subgroup on cpc_subgroup.id = cpc_current.subgroup_id where id is null;"]
select_list=["SELECT distinct patent_id FROM rawassignee LEFT JOIN patent on patent.id = patent_id where patent.id is null limit 5;","SELECT distinct patent_id FROM rawassignee LEFT JOIN patent on patent.id = patent_id where patent.id is null limit 5;","SELECT pct_data.patent_id FROM pct_data LEFT JOIN patent on patent.id = pct_data.patent_id where patent.id is null limit 5;",
             "SELECT distinct rawlocation.location_id FROM location_assignee LEFT JOIN rawlocation on rawlocation.location_id_transformed = location_assignee.location_id LEFT JOIN location on location.id = rawlocation.location_id where assignee_id = '' limit 5;","SELECT * FROM cpc_current LEFT JOIN cpc_subsection on cpc_subsection.id = cpc_current.subsection_id where id is null limit 100;","SELECT distinct location_assignee.location_id FROM temp_patent_list LEFT JOIN patent_assignee on patent_assignee.patent_id = temp_patent_list.id LEFT JOIN location_assignee on location_assignee.assignee_id = patent_assignee.assignee_id LEFT JOIN rawlocation on rawlocation.location_id_transformed = location_assignee.location_id limit 5;","SELECT distinct location_inventor.location_id FROM temp_patent_list LEFT JOIN patent_inventor on patent_inventor.patent_id = temp_patent_list.id LEFT JOIN location_inventor on location_inventor.inventor_id = patent_inventor.inventor_id LEFT JOIN location on location.id = location_inventor.location_id where location.id is null limit 5;","SELECT distinct patent_id FROM rawinventor LEFT JOIN rawlocation on rawlocation.id = rawinventor.rawlocation_id where rawlocation.id is null limit 5;","SELECT distinct patent_id FROM cpc_current LEFT JOIN cpc_subgroup on cpc_subgroup.id = cpc_current.subgroup_id where id is null limit 5;"
,"SELECT distinct mainclass_id FROM uspc_current LEFT JOIN mainclass_current on mainclass_current.id = uspc_current.mainclass_id where title is null AND mainclass_id != 'No longer published' limit 5;"
,"SELECT distinct id FROM location_inventor LEFT JOIN location on location.id = location_inventor.location_id where location_inventor.location_id is null limit 5;"
,"SELECT distinct patent_id FROM nber LEFT JOIN nber_category on nber_category.id = nber.category_id where title is null limit 5;"
,"SELECT distinct category_id FROM nber LEFT JOIN nber_category on nber_category.id = nber.category_id where title is null limit 5;"
,"SELECT distinct patent_id FROM nber LEFT JOIN nber_subcategory on nber_subcategory.id = nber.subcategory_id where title is null limit 5;"
,"SELECT distinct subcategory_id FROM nber LEFT JOIN nber_subcategory on nber_subcategory.id = nber.subcategory_id where title is null limit 5;"
,"SELECT distinct patent_id FROM uspc LEFT JOIN mainclass on mainclass.id = uspc.mainclass_id where mainclass.id is null limit 5;"
,"SELECT distinct mainclass_id FROM uspc LEFT JOIN mainclass on mainclass.id = uspc.mainclass_id where patent_id is null limit 5;"
,"SELECT distinct patent_id FROM uspc LEFT JOIN subclass on subclass.id = uspc.mainclass_id where subclass.id is null limit 5;"
,"SELECT distinct subclass_id FROM uspc LEFT JOIN subclass on subclass.id = uspc.mainclass_id where patent_id is null limit 5;"
,"SELECT distinct wipo.patent_id FROM wipo LEFT JOIN wipo_field on wipo_field.id = wipo.field_id where wipo_field.id is null limit 5;"
,"SELECT distinct patent_id FROM cpc_current LEFT JOIN cpc_subgroup on cpc_subgroup.id = cpc_current.subgroup_id where id is null limit 5;"]

results = pd.read_excel("{0}/4_left_joins_nonstandard.xlsx".format(previous_qa_loc))
left_joins_counts = []
left_joins_queries = []
id_list=[]
for i in range(len(query_list)):
    print (query_list[i])
    conn = engine.connect()
    var = conn.execute(query_list[i])
    count = [row[0] for row in var][0]
    left_joins_counts.append(count)
    #print (var)
    #print ("---------------------------------")
    left_joins_queries.append(query_list[i])   
    if count >0: #ie if there are any rows in the left table not in the right 
        ids_sql_var = conn.execute(select_list[i])
        ids_sql=[row[0] for row in ids_sql_var][0]
        ids= ids_sql.values.T.tolist()
        id_list.append(ids[0])
    else:
        id_list.append('none')
conn.close()

SELECT count(distinct patent_id) FROM rawassignee LEFT JOIN patent on patent.id = patent_id where patent.id is null;
SELECT count(distinct patent_id) FROM rawassignee LEFT JOIN patent on patent.id = patent_id where patent.id is null;
SELECT count(pct_data.patent_id) FROM pct_data LEFT JOIN patent on patent.id = pct_data.patent_id where patent.id is null;
SELECT count(distinct rawlocation.location_id) FROM location_assignee LEFT JOIN rawlocation on rawlocation.location_id_transformed = location_assignee.location_id LEFT JOIN location on location.id = rawlocation.location_id where assignee_id = '';
SELECT count(*) FROM cpc_current LEFT JOIN cpc_subsection on cpc_subsection.id = cpc_current.subsection_id where id is null limit 100;
SELECT count(distinct location_assignee.location_id) FROM temp_patent_list LEFT JOIN patent_assignee on patent_assignee.patent_id = temp_patent_list.id LEFT JOIN location_assignee on location_assignee.assignee_id = patent_assignee.assignee_id LEFT JOIN rawlocat

Exception during reset or similar
Traceback (most recent call last):
  File "/opt/conda/lib/python3.6/site-packages/sqlalchemy/pool.py", line 709, in _finalize_fairy
    fairy._reset(pool)
  File "/opt/conda/lib/python3.6/site-packages/sqlalchemy/pool.py", line 880, in _reset
    pool._dialect.do_rollback(self)
  File "/opt/conda/lib/python3.6/site-packages/sqlalchemy/dialects/mysql/base.py", line 1805, in do_rollback
    dbapi_connection.rollback()
_mysql_exceptions.OperationalError: (2006, 'MySQL server has gone away')
Exception during reset or similar
Traceback (most recent call last):
  File "/opt/conda/lib/python3.6/site-packages/sqlalchemy/pool.py", line 709, in _finalize_fairy
    fairy._reset(pool)
  File "/opt/conda/lib/python3.6/site-packages/sqlalchemy/pool.py", line 880, in _reset
    pool._dialect.do_rollback(self)
  File "/opt/conda/lib/python3.6/site-packages/sqlalchemy/dialects/mysql/base.py", line 1805, in do_rollback
    dbapi_connection.rollback()
_mysql_exceptions.O

AttributeError: 'str' object has no attribute 'values'

In [ ]:
query_list=["SELECT count(distinct patent_id) FROM rawassignee LEFT JOIN patent on patent.id = patent_id where patent.id is null;",
            "SELECT count(distinct patent_id) FROM rawassignee LEFT JOIN patent on patent.id = patent_id where patent.id is null;",
            "SELECT count(pct_data.patent_id) FROM pct_data LEFT JOIN patent on patent.id = pct_data.patent_id where patent.id is null;",
            "SELECT count(distinct rawlocation.location_id) FROM location_assignee LEFT JOIN rawlocation on rawlocation.location_id_transformed = location_assignee.location_id LEFT JOIN location on location.id = rawlocation.location_id where assignee_id = '';",
            "SELECT count(*) FROM cpc_current LEFT JOIN cpc_subsection on cpc_subsection.id = cpc_current.subsection_id where id is null limit 100;",
            "SELECT count(distinct location_assignee.location_id) FROM temp_patent_list LEFT JOIN patent_assignee on patent_assignee.patent_id = temp_patent_list.id LEFT JOIN location_assignee on location_assignee.assignee_id = patent_assignee.assignee_id LEFT JOIN rawlocation on rawlocation.location_id_transformed = location_assignee.location_id;","SELECT count(distinct location_inventor.location_id) FROM temp_patent_list LEFT JOIN patent_inventor on patent_inventor.patent_id = temp_patent_list.id LEFT JOIN location_inventor on location_inventor.inventor_id = patent_inventor.inventor_id LEFT JOIN location on location.id = location_inventor.location_id where location.id is null;","SELECT count(distinct patent_id) FROM rawinventor LEFT JOIN rawlocation on rawlocation.id = rawinventor.rawlocation_id where rawlocation.id is null;",
            "SELECT count(distinct patent_id) FROM cpc_current LEFT JOIN cpc_subgroup on cpc_subgroup.id = cpc_current.subgroup_id where id is null;",
            "SELECT count(distinct mainclass_id) FROM uspc_current LEFT JOIN mainclass_current on mainclass_current.id = uspc_current.mainclass_id where title is null AND mainclass_id != 'No longer published';",
            "SELECT count(distinct id) FROM location_inventor LEFT JOIN location on location.id = location_inventor.location_id where location_inventor.location_id is null;","SELECT count(distinct patent_id) FROM nber LEFT JOIN nber_category on nber_category.id = nber.category_id where title is null;","SELECT count(distinct category_id) FROM nber LEFT JOIN nber_category on nber_category.id = nber.category_id where title is null;",
"SELECT count(distinct patent_id) FROM nber LEFT JOIN nber_subcategory on nber_subcategory.id = nber.subcategory_id where title is null;","SELECT count(distinct subcategory_id) FROM nber LEFT JOIN nber_subcategory on nber_subcategory.id = nber.subcategory_id where title is null;","SELECT count(distinct patent_id) FROM uspc LEFT JOIN mainclass on mainclass.id = uspc.mainclass_id where mainclass.id is null;",
            "SELECT count(distinct mainclass_id) FROM uspc LEFT JOIN mainclass on mainclass.id = uspc.mainclass_id where patent_id is null;","SELECT count(distinct patent_id) FROM uspc LEFT JOIN subclass on subclass.id = uspc.mainclass_id where subclass.id is null;",
            "SELECT count(distinct subclass_id) FROM uspc LEFT JOIN subclass on subclass.id = uspc.mainclass_id where patent_id is null;","SELECT count(distinct wipo.patent_id) FROM wipo LEFT JOIN wipo_field on wipo_field.id = wipo.field_id where wipo_field.id is null;",
            "SELECT count(distinct patent_id) FROM cpc_current LEFT JOIN cpc_subgroup on cpc_subgroup.id = cpc_current.subgroup_id where id is null;"]
select_list=["SELECT distinct patent_id FROM rawassignee LEFT JOIN patent on patent.id = patent_id where patent.id is null limit 5;","SELECT distinct patent_id FROM rawassignee LEFT JOIN patent on patent.id = patent_id where patent.id is null limit 5;","SELECT pct_data.patent_id FROM pct_data LEFT JOIN patent on patent.id = pct_data.patent_id where patent.id is null limit 5;",
             "SELECT distinct rawlocation.location_id FROM location_assignee LEFT JOIN rawlocation on rawlocation.location_id_transformed = location_assignee.location_id LEFT JOIN location on location.id = rawlocation.location_id where assignee_id = '' limit 5;","SELECT * FROM cpc_current LEFT JOIN cpc_subsection on cpc_subsection.id = cpc_current.subsection_id where id is null limit 100;","SELECT distinct location_assignee.location_id FROM temp_patent_list LEFT JOIN patent_assignee on patent_assignee.patent_id = temp_patent_list.id LEFT JOIN location_assignee on location_assignee.assignee_id = patent_assignee.assignee_id LEFT JOIN rawlocation on rawlocation.location_id_transformed = location_assignee.location_id limit 5;","SELECT distinct location_inventor.location_id FROM temp_patent_list LEFT JOIN patent_inventor on patent_inventor.patent_id = temp_patent_list.id LEFT JOIN location_inventor on location_inventor.inventor_id = patent_inventor.inventor_id LEFT JOIN location on location.id = location_inventor.location_id where location.id is null limit 5;","SELECT distinct patent_id FROM rawinventor LEFT JOIN rawlocation on rawlocation.id = rawinventor.rawlocation_id where rawlocation.id is null limit 5;","SELECT distinct patent_id FROM cpc_current LEFT JOIN cpc_subgroup on cpc_subgroup.id = cpc_current.subgroup_id where id is null limit 5;"
,"SELECT distinct mainclass_id FROM uspc_current LEFT JOIN mainclass_current on mainclass_current.id = uspc_current.mainclass_id where title is null AND mainclass_id != 'No longer published' limit 5;"
,"SELECT distinct id FROM location_inventor LEFT JOIN location on location.id = location_inventor.location_id where location_inventor.location_id is null limit 5;"
,"SELECT distinct patent_id FROM nber LEFT JOIN nber_category on nber_category.id = nber.category_id where title is null limit 5;"
,"SELECT distinct category_id FROM nber LEFT JOIN nber_category on nber_category.id = nber.category_id where title is null limit 5;"
,"SELECT distinct patent_id FROM nber LEFT JOIN nber_subcategory on nber_subcategory.id = nber.subcategory_id where title is null limit 5;"
,"SELECT distinct subcategory_id FROM nber LEFT JOIN nber_subcategory on nber_subcategory.id = nber.subcategory_id where title is null limit 5;"
,"SELECT distinct patent_id FROM uspc LEFT JOIN mainclass on mainclass.id = uspc.mainclass_id where mainclass.id is null limit 5;"
,"SELECT distinct mainclass_id FROM uspc LEFT JOIN mainclass on mainclass.id = uspc.mainclass_id where patent_id is null limit 5;"
,"SELECT distinct patent_id FROM uspc LEFT JOIN subclass on subclass.id = uspc.mainclass_id where subclass.id is null limit 5;"
,"SELECT distinct subclass_id FROM uspc LEFT JOIN subclass on subclass.id = uspc.mainclass_id where patent_id is null limit 5;"
,"SELECT distinct wipo.patent_id FROM wipo LEFT JOIN wipo_field on wipo_field.id = wipo.field_id where wipo_field.id is null limit 5;"
,"SELECT distinct patent_id FROM cpc_current LEFT JOIN cpc_subgroup on cpc_subgroup.id = cpc_current.subgroup_id where id is null limit 5;"]

def left_joins_weird(new_database, previous_qa_loc, new_qa_loc, query_list, select_list):
    '''new_db: new database
    previous_qa_loc: location of previous qa document
    new_qa_loc: location of new qa document
    this function does all left joins that don't follow the same pattern 
    in the new database and writes output to a csv file'''
    results = pd.read_excel("{0}/4_left_joins_nonstandard.xlsx".format(previous_qa_loc))
    left_joins_counts = []
    left_joins_queries = []
    id_list=[]
    for i in range(len(query_list)):
        print query_list[i]
        conn = conn.connect
        var = conn.execute(query_list[i])
        count = [row[0] for row in var][0]
        left_joins_counts.append(count)
        print var
        print "---------------------------------"
        left_joins_queries.append(query_list[i])   
        if count >0: #ie if there are any rows in the left table not in the right 
            ids_sql = conn.connect(select_list[i])
            ids= ids_sql.values.T.tolist()
            id_list.append(ids[0])
        else:
            id_list.append('none')
    results['{}_counts'.format(new_database)] = left_joins_counts
    results['{}_example_ids'.format(new_database)] = id_list
    return results
def write_wierd(results, new_qa_loc):
    results.to_csv("{0}/07_left_joins_nonstandard".format(new_qa_loc), index =False)

In [ ]:
nonstandard_results = left_joins_weird(new_db, previous_qa_loc, new_qa_loc, query_list, select_list)
write_weird(nonstandard_results, new_qa_loc)